In [14]:
import os 
from dotenv import load_dotenv, find_dotenv
_= load_dotenv(find_dotenv())
openai_api_key = os.getenv('OPENAI_API_KEY')

In [15]:
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(openai_api_key=openai_api_key, model_name="gpt-3.5-turbo-0125")

In [16]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

promt = ChatPromptTemplate.from_template("tell me a joke about {topic}")
chain = promt | llm | StrOutputParser()


In [17]:
chain.invoke({"topic": "bears"})

'Why did the bear break up with his girlfriend?\n\nBecause she was always grizzly.'

## Runnable Pass Through

It does not do anything to input data
Lets see it in a very simple example : a chain with just RunnablePassThrough() will output the original input without any modification.

In [18]:
from langchain_core.runnables import RunnablePassthrough
chain = RunnablePassthrough()

chain.invoke("Hello World!")

'Hello World!'

## Runnable Lambda

To use a custom function inside a LCEL chain we need to wrap it with RunnableLambda
Lets define a very simple function to create Russian lastnames.

In [19]:
def create_russian_lastname(first_name: str) -> str:
    return f"{first_name}ovich"

In [20]:
from langchain_core.runnables import RunnableLambda

chain = RunnablePassthrough() | RunnableLambda(create_russian_lastname)

In [21]:
chain.invoke("Ivan")

'Ivanovich'

## Runnable Parrarallel

We will use Runnable PArrallel to run tasks in parallel.
This is the most useful runnable in LCEL.

In the following chain , Runnable Parrallel will run two tasks in parallel:
operation_a will be RunnablePassthrough() and operation_b will be RunnableLambda(create_russian_lastname)
"""

In [22]:
from langchain_core.runnables import RunnableParallel

chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "operation_b": RunnableLambda(create_russian_lastname),
    }
)

In [23]:
chain.invoke("Ivan")

{'operation_a': 'Ivan', 'operation_b': 'Ivanovich'}

In [30]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("tell me a joke about {soccer_player}")
output_parser = StrOutputParser()

In [31]:
def russian_lastname_from_dic(person):
    return person["name"] + "ovich"

In [32]:
chain = RunnableParallel(
    {
        "operation_a": RunnablePassthrough(),
        "soccer_player": RunnableLambda(russian_lastname_from_dic),
        "operation_c": RunnablePassthrough(),
    }
) | prompt | llm | output_parser



In [33]:
chain.invoke({
    "name1":"Peter",
    "name": "Ivan"})

'Why did Ivanovich bring a ladder to the bar? \n\nBecause he heard the drinks were on the house!'

## Lets see a more advamced use of Runnable Parralel

In [ ]:
from langchain_community.vectorstores import Chroma
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Create vectorstore with embeddings
vectorstore = Chroma.from_texts(
    [
        "Introduction to AI: Artificial Intelligence (AI) represents the simulation of human intelligence in machines programmed to think and learn like humans. The field encompasses everything from basic rule-based systems to complex neural networks capable of processing natural language, recognizing patterns, and making decisions."
    ],
    embedding=OpenAIEmbeddings()  # Ensure OpenAI API key is set in your environment
)

# Define retriever
retriever = vectorstore.as_retriever()

# Define prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

# Define retriever chain
retriever_chain = RunnableParallel(
    {"context": retriever, "question": RunnablePassthrough()}
) | prompt | model | StrOutputParser()

# Test the chain
response = retriever_chain.invoke({"question": "What is AI?"})
print(response)


TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'

In [ ]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma

# Create vectorstore with embeddings
vectorstore = Chroma.from_texts(
    [
        "Introduction to AI: Artificial Intelligence (AI) represents the simulation of human intelligence in machines programmed to think and learn like humans. The field encompasses everything from basic rule-based systems to complex neural networks capable of processing natural language, recognizing patterns, and making decisions."
    ],
    embedding=OpenAIEmbeddings()  # Ensure OpenAI API key is set in your environment
)

# Define retriever
retriever = vectorstore.as_retriever()

# Define prompt template
template = """Answer the question based only on the following context:
{context}

Question: {question}

Options:
1. {option1}
2. {option2}
3. {option3}
"""

prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

# Define chain

retriever_chain = (
    RunnablePassthrough()
    | RunnableParallel(
        {
            "context": retriever,
            "question": RunnablePassthrough(),
            "option1": RunnablePassthrough(),
            "option2": RunnablePassthrough(),
            "option3": RunnablePassthrough(),
        }
    )
    | prompt
    | model
    | StrOutputParser()
)
# Run chain
retriever_chain.invoke(
    {
        "question": "What is AI?",
        "option1": "A type of computer",
        "option2": "A branch of computer science",
        "option3": "A new programming language",
    }
)
# Output
response = retriever_chain.invoke(
    {
        "question": itemgetter("output")(response) | retriever,
        "option1": itemgetter("output")(response),
        "option2": itemgetter("output")(response),
        "option3": itemgetter("output")(response),  
    }
    | prompt
    | model
    | StrOutputParser()
)
print(response)


TypeError: argument 'text': 'dict' object cannot be converted to 'PyString'